In [1]:
!git clone https://github.com/dvlab-research/LongLoRA.git

Cloning into 'LongLoRA'...
remote: Enumerating objects: 298, done.
remote: Counting objects: 100% (186/186), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 298 (delta 114), reused 112 (delta 65), pack-reused 112
Receiving objects: 100% (298/298), 9.71 MiB | 15.02 MiB/s, done.
Resolving deltas: 100% (159/159), done.


In [5]:
%cd LongLoRA

/workspace/LongLoRA


In [6]:
!python -m pip install --upgrade pip
!pip install -r requirements.txt -q
!pip install flash-attn --no-build-isolation -q

In [7]:
!pip install huggingface_hub -q
from huggingface_hub import notebook_login

notebook_login()

In [8]:
import wandb
wandb.login()

wandb: Currently logged in as: ronankmcgovern (trelis). Use `wandb login --relogin` to force relogin


True

In [24]:
# import os
# os.environ["WANDB_MODE"] = "offline"
# os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# !python supervised-fine-tune.py  \
#         --model_name_or_path meta-llama/Llama-2-7b-chat-hf \
#         --bf16 True \
#         --output_dir Trelis/Llama-2-7b-chat-hf-32k-adapters       \
#         --model_max_length 32768 \
#         --use_flash_attn True \
#         --data_path ./32k.json \
#         --low_rank_training True \
#         --num_train_epochs 3  \
#         --per_device_train_batch_size 1     \
#         --per_device_eval_batch_size 1     \
#         --gradient_accumulation_steps 1     \
#         --evaluation_strategy "no"     \
#         --save_strategy "steps"     \
#         --save_steps 0     \
#         --save_total_limit 2     \
#         --learning_rate 2e-5     \
#         --weight_decay 0.0     \
#         --warmup_steps 20     \
#         --lr_scheduler_type "constant_with_warmup"     \
#         --logging_steps 10     \
#         --deepspeed "ds_configs/stage2.json" \
#         --tf32 True \
#         --cache_dir ./

In [9]:
!torchrun --nproc_per_node=2 supervised-fine-tune.py  \
        --model_name_or_path meta-llama/Llama-2-7b-chat-hf \
        --bf16 True \
        --output_dir Trelis/Llama-2-7b-chat-hf-32k-adapters       \
        --model_max_length 32768 \
        --use_flash_attn True \
        --data_path ./32k_n1000.json \
        --low_rank_training True \
        --num_train_epochs 3  \
        --per_device_train_batch_size 1     \
        --per_device_eval_batch_size 1     \
        --gradient_accumulation_steps 8     \
        --evaluation_strategy "no"     \
        --save_strategy "steps"     \
        --save_steps 100     \
        --save_total_limit 10     \
        --learning_rate 2e-5     \
        --weight_decay 0.0     \
        --warmup_steps 20     \
        --lr_scheduler_type "constant_with_warmup"     \
        --logging_steps 1     \
        --deepspeed "ds_configs/stage2.json" \
        --tf32 True \
        --cache_dir ./

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
[2023-10-09 21:31:41,781] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2023-10-09 21:31:41,796] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2023-10-09 21:31:43,219] [INFO] [comm.py:637:init_distributed] cdb=None
[2023-10-09 21:31:43,219] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
[2023-10-09 21:31:43,227] [INFO] [comm.py:637:init_distributed] cdb=None
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:02<00:00,  1.49s/it]
Using pad_token, but it is not set yet.
Using pad_token, but it is not set yet.
A chassis and associated telecommunica

In [10]:
!python3 merge_lora_weights_and_save_hf_model.py \
        --base_model meta-llama/Llama-2-7b-chat-hf \
        --peft_model Trelis/Llama-2-7b-chat-hf-32k-adapters \
        --context_size 32768 \
        --save_path Trelis/Llama-2-7b-chat-hf-32k

base model meta-llama/Llama-2-7b-chat-hf
peft model Trelis/Llama-2-7b-chat-hf-32k-adapters
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:01<00:00,  1.20it/s]
Using pad_token, but it is not set yet.
/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:844: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
model-00001-of-00002.safetensors:   0%|             | 0.00/9.98G [00:00<?, ?B/s]
Upload 2 LFS files:   0%|                                 | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%| | 8.19k/9.98G [00:00<160:01:14, 17.3kB/s

model-00001-of-00002.safetensors:   0%|    | 180k/9.98G [00:00<8:10:20, 339kB/s]

model-00002-of-00002.safetensors:   0%|    | 123k/3.50G [00:00<4:11:47, 232kB/s]

model-00001-of-00002.safetensors:   0%|    | 2.05M/9.98G [00:00<44:26, 3.74MB/s]

model-00001-of-00002.safetensors:   0%|    | 9.04M/9.98G [00:01<11:58, 13.9MB/s]

model-00001-of-00002.safe

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [12]:
# In case the supervised-fine-tune script does not push everything to hub (which it really should).

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

path_to_push="./Trelis/Llama-2-7b-chat-hf-32k"
new_model_name="Trelis/Llama-2-7b-chat-hf-32k"

model_to_push = AutoModelForCausalLM.from_pretrained(
    path_to_push,
    device_map={"":0},
    torch_dtype=torch.bfloat16)

model_to_push.push_to_hub(new_model_name, use_auth_token=True, max_shard_size="10GB", safe_serialization=True)

tokenizer = AutoTokenizer.from_pretrained(path_to_push, trust_remote_code=True)
tokenizer.push_to_hub(new_model_name, use_auth_token=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:844: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:844: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


CommitInfo(commit_url='https://huggingface.co/Trelis/Llama-2-7b-chat-hf-32k/commit/bec36ee5fa85f88f1385bbdc92d94ce97a5b1bf7', commit_message='Upload tokenizer', commit_description='', oid='bec36ee5fa85f88f1385bbdc92d94ce97a5b1bf7', pr_url=None, pr_revision=None, pr_num=None)

wandb: Waiting for W&B process to finish... (success).
